In [1]:
%cd ../src/

/mnt/c/Users/Jacob/Desktop/prosjektoppgave/tcav_atari/src


In [2]:
import numpy as np

from concepts import concept_instances
from probing import train_probes
from utils import load_game_data, load_q_network_device

In [3]:
game_data = load_game_data()
q_network, device = load_q_network_device()
q_network.to('cpu')

QNetwork(
  (network): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=3136, out_features=512, bias=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=4, bias=True)
  )
)

In [4]:
hyperparams = {
    'lr': 0.0001,
    'batch_size': 64,
    'epochs': 300,
    'lambda_l1': 0.00001,
    'patience': 10
}

In [5]:
total_scores = []
for concept in concept_instances.values():
    print(concept.name)
    layer_scores = train_probes(game_data, q_network, concept, hyperparams)
    scores = list(layer_scores.values())
    total_scores.extend(scores)
    print(f"Average score over all layers: {np.mean(scores):.4f}")
print(np.mean(total_scores))

random (b)
Epoch 1/300 - Train loss: 0.2545 - Test loss: 0.2652 - Test score: -0.0234
Epoch 2/300 - Train loss: 0.2532 - Test loss: 0.2628 - Test score: -0.0234
Epoch 3/300 - Train loss: 0.2538 - Test loss: 0.2519 - Test score: -0.0165
Epoch 4/300 - Train loss: 0.2549 - Test loss: 0.2528 - Test score: 0.0041
Epoch 5/300 - Train loss: 0.2508 - Test loss: 0.2525 - Test score: -0.0134
Epoch 6/300 - Train loss: 0.2490 - Test loss: 0.2814 - Test score: 0.0195
Epoch 7/300 - Train loss: 0.2490 - Test loss: 0.2532 - Test score: -0.0431
Epoch 8/300 - Train loss: 0.2493 - Test loss: 0.2523 - Test score: -0.0239
Epoch 9/300 - Train loss: 0.2493 - Test loss: 0.2666 - Test score: -0.0156
Epoch 10/300 - Train loss: 0.2487 - Test loss: 0.2776 - Test score: -0.0182
Epoch 11/300 - Train loss: 0.2471 - Test loss: 0.2708 - Test score: -0.0143
Epoch 12/300 - Train loss: 0.2487 - Test loss: 0.2551 - Test score: 0.0096
Epoch 13/300 - Train loss: 0.2472 - Test loss: 0.2604 - Test score: 0.0087
Epoch 14/300 -

: 